In [22]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime, timedelta
import firebase_admin
from firebase_admin import credentials, storage, db

In [35]:
# Initialize MT5 connection
def initialize_mt5():
    if not mt5.initialize():
        print(f"MT5 Initialize failed, error: {mt5.last_error()}")
        return False
    print(f"Connected to MT5. Account Info: {mt5.account_info()}")
    return True

# Retrieve data for a specific symbol and day with level added
def get_day_data_with_level(symbol, day, level):
    start = datetime(day.year, day.month, day.day)
    end = start + timedelta(days=1)
    rates = mt5.copy_rates_range(symbol, mt5.TIMEFRAME_D1, start, end)
    if rates is None or len(rates) == 0:
        return None
    highest = max(rate['high'] for rate in rates)
    lowest = min(rate['low'] for rate in rates)
    return {"Date": start.date(), "Symbol": symbol, "Highest": highest, "Lowest": lowest, "Level": level}

# Retrieve data for a symbol over multiple days, adding level
def get_symbol_data_with_level(symbol):
    levels = {0: "C", 1: "Y", 2: "V"}  # Map days_back to Level
    data = []
    today = datetime.now()
    for i, level in levels.items():
        day = today - timedelta(days=i)
        day_data = get_day_data_with_level(symbol, day, level)
        if day_data:
            data.append(day_data)
        else:
            print(f"Missing data for {symbol} on {day.date()}")
    return data

# Fetch data for all symbols with level column
def fetch_all_data_with_level(symbols):
    all_data = []
    for symbol in symbols:
        print(f"Fetching data for {symbol}...")
        symbol_data = get_symbol_data_with_level(symbol)
        if symbol_data:
            all_data.extend(symbol_data)
        else:
            print(f"No data found for {symbol}.")
    return pd.DataFrame(all_data)

# Initialize Firebase Admin SDK
def save_to_firebase(data, firebase_json_path, db_url):
    # Initialize Firebase Admin SDK if not already initialized
    try:
        app = firebase_admin.get_app()
        print(f"Using existing Firebase app: {app.name}")
    except ValueError:
        cred = credentials.Certificate(firebase_json_path)
        app = firebase_admin.initialize_app(cred, {
            "databaseURL": db_url
        })
        print(f"Firebase app initialized: {app.name}")
    
    # Push data to Firebase Realtime Database
    try:
        ref = db.reference('live data')  # Use a node called 'live_data'
        ref.set(data.to_dict(orient='records'))  # Convert data to dictionary and save
        print(f"Data successfully saved to Firebase Realtime Database.")
    except Exception as e:
        print(f"Error during saving data: {e}")


def main():
    symbols = [
        "XAUUSD", "BTCUSD", "BTCEUR", "ETHUSD", "SOLUSD", "XRPUSD", 
        "DSHUSD", "XAUEUR", "GBPAUD", "GBPNZD", "CHFJPY", 
        "EURJPY", "USA30", "USA100", "USOIL", "UKOIL", "USDJPY", 
        "GBPJPY", "USA500", "UK100", "CADJPY", "AUDJPY", "GBPCHF", 
        "GBPCAD", "NZDJPY", "GBPUSD", "EURGBP"
    ]

    if not initialize_mt5():
        return

    # Fetch data for all symbols
    data = fetch_all_data_with_level(symbols)
    if data.empty:
        print("No data fetched.")
        return

    # Firebase configuration
    firebase_json_path = "C:/Users/Shaheera/Documents/utp related/internship things/trading/live-data-a84f1-firebase-adminsdk-fbsvc-188ce70493.json"  # Replace with your Firebase credentials path
    db_url = "https://live-data-a84f1-default-rtdb.firebaseio.com/"  # Ensure this URL is correct

    # Save data to Firebase Realtime Database
    save_to_firebase(data, firebase_json_path, db_url)

In [36]:
# Run the script
if __name__ == "__main__":
    main()

Connected to MT5. Account Info: AccountInfo(login=48690778, trade_mode=0, leverage=1000, limit_orders=300, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=False, balance=100000.0, credit=0.0, profit=0.0, equity=100000.0, margin=0.0, margin_free=100000.0, margin_level=0.0, margin_so_call=50.0, margin_so_so=20.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='#shey - Nur Shaheera Husna Mohd Shahril', server='HFMarketsGlobal-Demo', currency='USD', company='HF Markets (SV) Ltd.')
Fetching data for XAUUSD...
Missing data for XAUUSD on 2025-01-26
Missing data for XAUUSD on 2025-01-25
Fetching data for BTCUSD...
Missing data for BTCUSD on 2025-01-26
Missing data for BTCUSD on 2025-01-25
Fetching data for BTCEUR...
Missing data for BTCEUR on 2025-01-26
Missing data for BTCEUR on 2025-01-25
Fetching data for ETHUSD...
Missing data for ETHUSD on 2025-01-26
Missing data for ETHUSD on 2025